## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_to_load = os.path.join("..", "Weather_Database/WeatherPy_Database.csv")
city_data_df = pd.read_csv(weather_data_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Margate,GB,2021-08-01 22:34:26,51.3813,1.3862,60.69,83,97,1.99,overcast clouds
1,1,Los Llanos De Aridane,ES,2021-08-01 22:36:59,28.6585,-17.9182,69.12,68,75,19.57,broken clouds
2,2,Punta Arenas,CL,2021-08-01 22:32:14,-53.1500,-70.9167,42.91,75,40,17.27,scattered clouds
3,3,Ballina,AU,2021-08-01 22:41:00,-28.8667,153.5667,67.96,77,90,10.36,overcast clouds
4,4,Ribeira Grande,PT,2021-08-01 22:34:00,38.5167,-28.7000,73.18,94,63,13.49,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Margate,GB,2021-08-01 22:34:26,51.3813,1.3862,60.69,83,97,1.99,overcast clouds
1,1,Los Llanos De Aridane,ES,2021-08-01 22:36:59,28.6585,-17.9182,69.12,68,75,19.57,broken clouds
3,3,Ballina,AU,2021-08-01 22:41:00,-28.8667,153.5667,67.96,77,90,10.36,overcast clouds
4,4,Ribeira Grande,PT,2021-08-01 22:34:00,38.5167,-28.7000,73.18,94,63,13.49,broken clouds
6,6,Jinchang,CN,2021-08-01 22:38:00,38.4953,102.1739,73.36,37,79,3.22,broken clouds
11,11,Port Elizabeth,ZA,2021-08-01 22:31:54,-33.9180,25.5701,61.09,72,75,29.93,broken clouds
12,12,East London,ZA,2021-08-01 22:35:29,-33.0153,27.9116,62.44,63,31,19.48,scattered clouds
15,15,Albany,US,2021-08-01 22:35:32,42.6001,-73.9662,71.47,84,89,6.73,moderate rain
17,17,Valparaiso,CL,2021-08-01 22:43:06,-33.0393,-71.6273,75.43,55,0,9.22,clear sky
18,18,Boyuibe,BO,2021-08-01 22:43:06,-20.4167,-63.2833,67.91,36,79,1.23,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID            313
City               313
Country            312
Date               313
Lat                313
Lng                313
Max Temp           313
Humidity           313
Cloudiness         313
Wind Speed         313
Current Weather    313
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Margate,GB,60.69,overcast clouds,51.3813,1.3862,
1,Los Llanos De Aridane,ES,69.12,broken clouds,28.6585,-17.9182,
3,Ballina,AU,67.96,overcast clouds,-28.8667,153.5667,
4,Ribeira Grande,PT,73.18,broken clouds,38.5167,-28.7000,
6,Jinchang,CN,73.36,broken clouds,38.4953,102.1739,
11,Port Elizabeth,ZA,61.09,broken clouds,-33.9180,25.5701,
12,East London,ZA,62.44,scattered clouds,-33.0153,27.9116,
15,Albany,US,71.47,moderate rain,42.6001,-73.9662,
17,Valparaiso,CL,75.43,clear sky,-33.0393,-71.6273,
18,Boyuibe,BO,67.91,broken clouds,-20.4167,-63.2833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotels = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Hotel_Database.csv"
# 8b. Export the City_Data into a csv
clean_hotels.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weahter</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotels[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots.

fig = gmaps.figure(display_errors=False, center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))